In [1]:
pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from google.colab import files
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
import pycountry
import warnings
warnings.filterwarnings('ignore', category=FutureWarning, module='sklearn.cluster._kmeans')

# 1. CARREGAR OS DADOS
uploaded = files.upload(); file_name = list(uploaded.keys())[0]
try:
    df = pd.read_excel(file_name)
except Exception as e:
    df = pd.read_csv(file_name)

# 2. LIMPEZA E PREPARAÇÃO
df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce'); df.dropna(subset=['Start Date'], inplace=True)
df_prob_city = df['Capital city'].value_counts().reset_index(); df_prob_city.columns = ['Capital', 'Probabilidade (Contagem)']
df_imp_city = df[df['Protests with a significant outcome'] == 'X']['Capital city'].value_counts().reset_index(); df_imp_city.columns = ['Capital', 'Impacto (Contagem de Outcomes)']
df_matrix_city = pd.merge(df_prob_city, df_imp_city, on='Capital', how='left').fillna(0)
df_matrix_city['Nivel_Risco'] = df_matrix_city['Probabilidade (Contagem)'] + df_matrix_city['Impacto (Contagem de Outcomes)']
cols_to_convert = ['Economic motivation?', 'Political motivation?', 'Corruption motivation?', 'Gender motivation?', 'Protests with a significant outcome', 'Violent government response', 'Long protests (more than 3 months)', 'Large protests (Over 100,000 protesting)']
for col in cols_to_convert: df[col] = df[col].apply(lambda x: 1 if x == 'X' else 0)
fill_values = {'Size category': 'Unknown', 'Freedom Rating (Status)': 'Unknown', 'Capital city': 'Unknown', 'Duration': 'Unknown', 'Key Participants': 'Unknown'}
df.fillna(value=fill_values, inplace=True)

# 3. GRÁFICO 1: MAPEAMENTO DE PAÍSES
country_counts = df['Country'].value_counts().reset_index(); country_counts.columns = ['Country', 'Total de Eventos']
fig1 = px.choropleth(country_counts, locations="Country", locationmode='country names', color="Total de Eventos", hover_name="Country", color_continuous_scale=px.colors.sequential.YlOrRd)
fig1.update_layout(title=None, margin=dict(l=0, r=0, t=40, b=0)); fig1.show()

# 4. GRÁFICO 2: MATRIZ DE RISCO (TREEMAP)
df_matrix_plot = df_matrix_city[df_matrix_city['Capital'] != 'Unknown']
fig2 = px.treemap(df_matrix_plot, path=[px.Constant("Capitais"), 'Capital'], values='Nivel_Risco', color='Nivel_Risco', color_continuous_scale='Reds', hover_data=['Probabilidade (Contagem)', 'Impacto (Contagem de Outcomes)'], title=None)
fig2.update_layout(margin=dict(l=0, r=0, t=40, b=0)); fig2.show()

# 5. LÓGICA GRÁFICOS ML (3 E 4)
def get_iso_alpha(country_name):
    try:
        return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
    except Exception:
        return None
df_city_ts = df[df['Capital city'] != 'Unknown'].groupby([pd.Grouper(key='Start Date', freq='ME'), 'Capital city']).size().reset_index(name='Contagem')
df_pivot = df_city_ts.pivot(index='Start Date', columns='Capital city', values='Contagem').fillna(0)

# 6. GRÁFICO 3 (ML): MAPA DE SIMILARIDADE
K_CLUSTERS = 5
if df_pivot.shape[1] >= K_CLUSTERS:
    try:
        data_for_clustering = df_pivot.T; kmeans = KMeans(n_clusters=K_CLUSTERS, random_state=42, n_init=10)
        df_clusters = pd.DataFrame({'Capital': data_for_clustering.index, 'Cluster_ID': kmeans.fit_predict(data_for_clustering)})
        cluster_map = {0: 'Perfil 1', 1: 'Perfil 2', 2: 'Perfil 3', 3: 'Perfil 4', 4: 'Perfil 5'}; df_clusters['Perfil_de_Risco_Temporal'] = df_clusters['Cluster_ID'].map(cluster_map).astype(str)
        df_capital_to_country = df[df['Capital city'] != 'Unknown'][['Capital city', 'Country']].drop_duplicates(); df_nodes = pd.merge(df_prob_city, df_clusters, on='Capital', how='inner'); df_nodes = pd.merge(df_nodes, df_capital_to_country, left_on='Capital', right_on='Capital city', how='inner'); df_nodes['Pais_ISO'] = df_nodes['Country'].apply(get_iso_alpha)
        if not df_nodes.empty and not df_nodes['Pais_ISO'].isnull().all():
            perfil_order = ['Perfil 1', 'Perfil 2', 'Perfil 3', 'Perfil 4', 'Perfil 5']
            fig3 = px.scatter_geo(df_nodes, locations="Pais_ISO", locationmode="ISO-3", color='Perfil_de_Risco_Temporal', size='Probabilidade (Contagem)', hover_name='Capital', hover_data={'Perfil_de_Risco_Temporal': True, 'Probabilidade (Contagem)': True, 'Country': True, 'Pais_ISO': False}, projection="natural earth", title=None, color_discrete_map={'Perfil 1': '#fee5d9', 'Perfil 2': '#fcae91', 'Perfil 3': '#fb6a4a', 'Perfil 4': '#de2d26', 'Perfil 5': '#a50f15'}, category_orders={'Perfil_de_Risco_Temporal': perfil_order})
            fig3.update_layout(legend_title="Perfil de Risco (ML)", margin=dict(l=0, r=0, t=40, b=0)); fig3.show()
    except Exception as e: pass

# 7. LÓGICA PREDITIVA (PARA CSV E GRÁFICO 4)
predictions = []
motiv_map = {'Econômica': 'Economic motivation?', 'Política': 'Political motivation?', 'Corrupção': 'Corruption motivation?', 'Gênero': 'Gender motivation?'}
outco_map = {'Outcome Significativo': 'Protests with a significant outcome', 'Resposta Gov. Violenta': 'Violent government response', 'Protesto Longo (>3m)': 'Long protests (more than 3 months)', 'Protesto Grande (>100k)': 'Large protests (Over 100,000 protesting)'}

for city in df[df['Capital city'] != 'Unknown']['Capital city'].unique():
    df_city = df[df['Capital city'] == city].copy()
    country_name = df_city['Country'].mode().iloc[0]; country_iso = get_iso_alpha(country_name); freedom = df_city['Freedom Rating (Status)'].mode().iloc[0]; size = df_city['Size category'].mode().iloc[0]; participants = df_city['Key Participants'].mode().iloc[0]; duration = df_city['Duration'].mode().iloc[0]
    motivations_str = ', '.join([k for k, v in motiv_map.items() if df_city[v].sum() > 0]) or 'Não especificada'
    outcomes_str = ', '.join([k for k, v in outco_map.items() if df_city[v].sum() > 0]) or 'Não especificado'
    df_ts = df_city.set_index('Start Date').resample('YE').size().reset_index(name='Contagem'); df_ts['Ano'] = df_ts['Start Date'].dt.year
    tendencia = "Sem Dados"
    if len(df_ts) >= 2:
        X = df_ts[['Ano']]; y = df_ts['Contagem']; model = LinearRegression().fit(X, y); coef = model.coef_[0]
        tendencia = "Risco Muito Elevado" if coef > 0.5 else "Risco Elevado" if coef > 0.1 else "Risco Médio" if coef >= -0.1 else "Risco Baixo" if coef >= -0.5 else "Risco Muito Baixo"
    justificativa = f"Cidade: {city} (País: {country_name}, Nível Liberdade: {freedom}). Tendência: '{tendencia}'. Motivações: ({motivations_str}). Participantes: ({participants}). Duração: '{duration}'. Tamanho: '{size}'. Resultados comuns: {outcomes_str}."
    predictions.append({'Capital': city, 'Pais_ISO': country_iso, 'Pais_Nome': country_name, 'Tendencia_5_Anos': tendencia, 'Nivel_Liberdade_Mio': freedom, 'Tamanho_Medio_Historico': size, 'Duracao_Media_Historica': duration, 'Principais_Participantes': participants, 'Principais_Motivações': motivations_str, 'Resultados_Comuns': outcomes_str, 'Perfil_Risco_Justificativa': justificativa})

# 8. GERAR CSV E GRÁFICO 4
df_pred = pd.DataFrame(predictions)
csv_filename = 'previsao_risco_cidades.csv'; df_pred.to_csv(csv_filename, index=False, encoding='utf-8-sig'); files.download(csv_filename); print(f"Arquivo '{csv_filename}' baixado!")
df_map_data = df_pred.merge(df_matrix_city, on='Capital', how='left')
tendencia_order = ['Risco Muito Elevado', 'Risco Elevado', 'Risco Médio', 'Risco Baixo', 'Risco Muito Baixo', 'Sem Dados']
fig4 = px.scatter_geo(
    df_map_data, locations="Pais_ISO", locationmode="ISO-3",
    color="Tendencia_5_Anos", size="Probabilidade (Contagem)",
    hover_name="Capital",
    hover_data=["Pais_Nome", "Tendencia_5_Anos", "Nivel_Liberdade_Mio", "Principais_Motivações"],
    projection="natural earth",
    title=None,
    color_discrete_map={
        'Risco Muito Elevado': '#a50f15', 'Risco Elevado': '#de2d26',
        'Risco Médio': '#fb6a4a', 'Risco Baixo': '#fcae91',
        'Risco Muito Baixo': '#fee5d9', 'Sem Dados': '#bdbdbd'
    },
    category_orders={'Tendencia_5_Anos': tendencia_order}
)
fig4.update_layout(margin=dict(l=0, r=0, t=40, b=0), legend_title="Previsão de Tendência"); fig4.show()

Saving gpt - Copia.xlsx to gpt - Copia.xlsx


/tmp/ipython-input-2021239772.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce'); df.dropna(subset=['Start Date'], inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo 'previsao_risco_cidades.csv' baixado!
